In [ ]:
!pip install git+https://github.com/huggingface/transformers
!pip install datasets
!pip install bitsandbytes
!pip install peft==0.4.0 trl

  Cloning https://github.com/huggingface/transformers to /private/var/folders/9c/h1vdj_9x2kg1_99d25730yd80000gn/T/pip-req-build-h7k1y85s
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /private/var/folders/9c/h1vdj_9x2kg1_99d25730yd80000gn/T/pip-req-build-h7k1y85s
  Resolved https://github.com/huggingface/transformers to commit 6dfd561d9cd722dfc09f702355518c6d09b9b4e3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 6.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 6.7 MB/s eta 0:00:00a 0:00:01
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Using cached fil

In [3]:
pip install trl

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install tf-keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 5.6 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [11]:
!pip install --upgrade bitsandbytes==0.43.1

ERROR: Could not find a version that satisfies the requirement bitsandbytes==0.43.1 (from versions: 0.31.8, 0.32.0, 0.32.1, 0.32.2, 0.32.3, 0.33.0, 0.33.1, 0.34.0, 0.35.0, 0.35.1, 0.35.2, 0.35.3, 0.35.4, 0.36.0, 0.36.0.post1, 0.36.0.post2, 0.37.0, 0.37.1, 0.37.2, 0.38.0, 0.38.0.post1, 0.38.0.post2, 0.38.1, 0.39.0, 0.39.1, 0.40.0, 0.40.0.post1, 0.40.0.post2, 0.40.0.post3, 0.40.0.post4, 0.40.1, 0.40.1.post1, 0.40.2, 0.41.0, 0.41.1, 0.41.2, 0.41.2.post1, 0.41.2.post2, 0.41.3, 0.41.3.post1, 0.41.3.post2, 0.42.0)
ERROR: No matching distribution found for bitsandbytes==0.43.1


In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from datasets import load_dataset
from trl import SFTTrainer
import torch
import json
import pandas as pd

'NoneType' object has no attribute 'cadam32bit_grad_fp32'


/opt/anaconda3/envs/tfenv/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [17]:
pip install transformers accelerate

Note: you may need to restart the kernel to use updated packages.


In [21]:
from transformers import AutoModelForCausalLM, AutoTokenizer

MODEL_ID = "microsoft/Phi-3-mini-4k-instruct"

max_seq_length = 2048
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    trust_remote_code=True,
    torch_dtype="auto",  # will use float32 on CPU by default
)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_ID,
    trust_remote_code=True,
    max_seq_length=max_seq_length
)

tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 33.95it/s]


In [23]:
df_train = pd.read_csv("../data/Customer-Support/Bitext_Sample_Customer_Service_Training_Dataset.csv",
                      usecols=['utterance','intent'])

df_train.head(3)

,utterance,intent
0,would it be possible to cancel the order I made?,cancel_order
1,cancelling order,cancel_order
2,I need assistance canceling the last order I h...,cancel_order


In [25]:
df_test = pd.read_csv("../data/Customer-Support/Bitext_Sample_Customer_Service_Testing_Dataset.csv",
                      usecols=['utterance','intent'])

LABELS  = df_test['intent'].unique()
df_test.head(3)

,utterance,intent
0,I have a question about cancelling an order,cancel_order
1,help canceling the order I have made,cancel_order
2,I do not know how to cancel an order I have made,cancel_order


In [28]:
LABELS

array(['cancel_order', 'change_order', 'change_shipping_address',
       'check_cancellation_fee', 'check_invoice', 'check_payment_methods',
       'check_refund_policy', 'complaint', 'contact_customer_service',
       'contact_human_agent', 'create_account', 'delete_account',
       'delivery_options', 'delivery_period', 'edit_account',
       'get_invoice', 'get_refund', 'newsletter_subscription',
       'payment_issue', 'place_order', 'recover_password',
       'registration_problems', 'review', 'set_up_shipping_address',
       'switch_account', 'track_order', 'track_refund'], dtype=object)

In [29]:
from transformers import pipeline

system_prompt = """ 
<|system|>
You are a AI powered Customer Care Assistant who classifies incoming user queries to one of the
following intents:

- cancel_order
- change_order
- change_shipping_address
- check_cancellation_fee
- check_invoice
- check_payment_methods
- check_refund_policy
- complaint
- contact_customer_service
- contact_human_agent
- create_account
- delete_account
- delivery_options
- delivery_period
- edit_account
- get_invoice
- get_refund
- newsletter_subscription
- payment_issue
- place_order
- recover_password
- registration_problems
- review
- set_up_shipping_address
- switch_account
- track_order
- track_refund


You output should be one of the above intents as output and no other text.
Do not make up any answer.
Do not provide reasoning or explanations.

<|end|>
"""

pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
    )


def get_response(text):
    user_prompt = f"""
    <|user|>
    {text}
    <|end|>
    <|assistant|>
    """

    prompt = system_prompt + '\n'+ user_prompt
    

    generation_args = {
        "max_new_tokens": 100,
        "return_full_text": False,
        "temperature": 0.0,
        "do_sample": False,
    }

    output = pipe(prompt, **generation_args)
    output = output[0]['generated_text']
    
    
    
    return output
    

Device set to use mps:0


In [32]:
from difflib import get_close_matches

def closeMatches(word):
    """
    find closest text
    """
    out = get_close_matches(word, LABELS, n=1,cutoff=0.5)    
    return out[0] if len(out)>0 else word


def predict_frame(df,model,verbose=False):
     
    
    predicted_text = []
    for i in range(df.shape[0]):
        
        record = df.iloc[i]
        text = record['utterance']
        actual_intent = record['intent']
   
        predicted_intent = get_response(text)
        predicted_text.append(predicted_intent.strip())
        
        if verbose:
            print(f"Actual text:{text}")
            print(f"Actual intent: {actual_intent}")
            print(f"Predicted intent: {predicted_intent}")
            
        
  
    df['predicted_text'] = predicted_text
    df['predicted_intent'] = df['predicted_text'].apply(lambda x: closeMatches(x) if x not in LABELS else x)
  
    return df
   
    
df_test_predicted = predict_frame(df_test,model,verbose=False)

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


AttributeError: 'DynamicCache' object has no attribute 'seen_tokens'